In [20]:
from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("16"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Reindeer Maze

[Open Website](https://adventofcode.com/2024/day/16)

In [21]:
# helper functions
def domain_from_input(input: InputReader) -> list:
    lines = input.matrix()

    return lines


test_input = domain_from_input(puzzle.example(0))
test_input.print()

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############


In [22]:
from common.matrix import Direction, MatrixNavigator


# test case (part 1)
def part_1(reader: InputReader, debug: bool) -> int:
    matrix = domain_from_input(reader)

    if debug:
        matrix.print()

    queue = []
    # find start
    for x, y, value in matrix:
        if value == 'S':
            queue.append((x, y, 0, Direction.RIGHT, []))
            break

    def new_directions(direction):
        if direction == Direction.UP:
            return [Direction.LEFT, Direction.RIGHT]
        if direction == Direction.DOWN:
            return [Direction.LEFT, Direction.RIGHT]
        if direction == Direction.LEFT:
            return [Direction.UP, Direction.DOWN]
        if direction == Direction.RIGHT:
            return [Direction.UP, Direction.DOWN]

    # perform a BFS to find the shortest path
    while queue:
        # find node with the lowest cost
        for i in range(1, len(queue)):
            if queue[i][2] < queue[0][2]:
                queue[0], queue[i] = queue[i], queue[0]
        # pop the node
        x, y, cost, last_direction, history = queue.pop(0)

        pointer = MatrixNavigator(matrix, x, y)
        if pointer.get_value() == 'E':
            return cost

        # if we have been here before, skip
        if (x, y) in history:
            continue
        history.append((x, y))

        # check if we can move forward
        ok, value = pointer.peek_value(last_direction)
        if ok and value != '#':
            new_pointer = pointer.copy()
            new_pointer.move(last_direction)
            dx, dy = new_pointer.get_position()
            queue.append((dx, dy, cost + 1, last_direction, history))

        # check if we can turn
        for direction in new_directions(last_direction):
            ok, value = pointer.peek_value(direction)
            if ok and value != '#':
                new_pointer = pointer.copy()
                new_pointer.move(direction)
                dx, dy = new_pointer.get_position()
                new_history = history.copy()
                new_history.append((dx, dy))
                queue.append((dx, dy, cost + 1001, direction, history))

    return 0


result = part_1(puzzle.get_code_block(0), True)
display(result)
assert result == 7036

result = part_1(puzzle.get_code_block(2), True)
display(result)
assert result == 11048

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############


7036

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################


11048

In [23]:
# real case (part 1)
result = part_1(puzzle.input(), False)
display(result)
assert result == 115500

115500

In [24]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    matrix = domain_from_input(reader)

    if debug:
        matrix.print()
        print()

    queue = []
    # find start
    for x, y, value in matrix:
        if value == 'S':
            queue.append(((x, y), 0, Direction.RIGHT, []))
            break

    def new_directions(direction):
        if direction == Direction.UP:
            return [Direction.LEFT, Direction.RIGHT]
        if direction == Direction.DOWN:
            return [Direction.LEFT, Direction.RIGHT]
        if direction == Direction.LEFT:
            return [Direction.UP, Direction.DOWN]
        if direction == Direction.RIGHT:
            return [Direction.UP, Direction.DOWN]

    solution_points = []
    visited = {}
    solution = None

    # perform a BFS to find the shortest path
    while queue:
        # find node with the lowest cost
        for i in range(1, len(queue)):
            if queue[i][1] < queue[0][1]:
                queue[0], queue[i] = queue[i], queue[0]
        (x, y), cost, last_direction, path = queue.pop(0)

        path = path.copy()
        path.append((x, y))

        # check for end
        pointer = MatrixNavigator(matrix, x, y)
        if pointer.get_value() == 'E':
            if solution is None:
                solution = cost

            if cost == solution:
                for point in path:
                    solution_points.append(point)
            continue

        # check for visited
        current_situation = ((x, y), last_direction)
        if current_situation in visited:
            if visited[current_situation] < cost:
                continue
        visited[current_situation] = cost

        # check if we can move forward
        ok, value = pointer.peek_value(last_direction)
        if ok and value != '#':
            new_pointer = pointer.copy()
            new_pointer.move(last_direction)
            queue.append((new_pointer.get_position(), cost + 1, last_direction, path))

        # check if we can turn
        for direction in new_directions(last_direction):
            ok, value = pointer.peek_value(direction)
            if ok and value != '#':
                new_pointer = pointer.copy()
                new_pointer.move(direction)
                queue.append((new_pointer.get_position(), cost + 1001, direction, path))

    # join all the solutions
    if debug:
        for point in solution_points:
            matrix.set_value(point[0], point[1], 'O')
        matrix.print()

    return len(set(solution_points))


result = part_2(puzzle.get_code_block(0), True)
display(result)
assert result == 45

result = part_2(puzzle.get_code_block(2), True)
display(result)
assert result == 64

###############
#.......#....E#
#.#.###.#.###.#
#.....#.#...#.#
#.###.#####.#.#
#.#.#.......#.#
#.#.#####.###.#
#...........#.#
###.#.#####.#.#
#...#.....#.#.#
#.#.#.###.#.#.#
#.....#...#.#.#
#.###.#.#.#.#.#
#S..#.....#...#
###############

###############
#.......#....O#
#.#.###.#.###O#
#.....#.#...#O#
#.###.#####.#O#
#.#.#.......#O#
#.#.#####.###O#
#..OOOOOOOOO#O#
###O#O#####O#O#
#OOO#O....#O#O#
#O#O#O###.#O#O#
#OOOOO#...#O#O#
#O###.#.#.#O#O#
#O..#.....#OOO#
###############


45

#################
#...#...#...#..E#
#.#.#.#.#.#.#.#.#
#.#.#.#...#...#.#
#.#.#.#.###.#.#.#
#...#.#.#.....#.#
#.#.#.#.#.#####.#
#.#...#.#.#.....#
#.#.#####.#.###.#
#.#.#.......#...#
#.#.###.#####.###
#.#.#...#.....#.#
#.#.#.#####.###.#
#.#.#.........#.#
#.#.#.#########.#
#S#.............#
#################

#################
#...#...#...#..O#
#.#.#.#.#.#.#.#O#
#.#.#.#...#...#O#
#.#.#.#.###.#.#O#
#OOO#.#.#.....#O#
#O#O#.#.#.#####O#
#O#O..#.#.#OOOOO#
#O#O#####.#O###O#
#O#O#..OOOOO#OOO#
#O#O###O#####O###
#O#O#OOO#..OOO#.#
#O#O#O#####O###.#
#O#O#OOOOOOO..#.#
#O#O#O#########.#
#O#OOO..........#
#################


64

In [25]:
# real case (part 2)
result = part_2(puzzle.input(), False)
display(result)
assert result == 679

679

In [26]:
# print easters eggs
puzzle.print_easter_eggs()

## Easter Eggs

<span title="I would say it's like Reindeer Golf, but knowing Reindeer, it's almost certainly nothing like Reindeer Golf.">lowest score</span> (I would say it's like Reindeer Golf, but knowing Reindeer, it's almost certainly nothing like Reindeer Golf.)